In [1]:
import warnings 
warnings.simplefilter("ignore")
import numpy as np
import pandas as pd
#from statsforecast.core import StatsForecast
#from statsforecast.models import AutoARIMA, Naive
from hierarchicalforecast.core import HierarchicalReconciliation
#from hierarchicalforecast.evaluation import evaluate
from hierarchicalforecast.methods import BottomUp, TopDown, MinTrace, TopDownSparse
#from utilsforecast.losses import mseaoaoa

# BU AND MINTRACE

In [5]:
%%time
path  = 'D:/yjz/Extension for hts/JayCode/Data/'

Y_ets_tr  = pd.read_pickle("h1_AutoETSfitt.pkl")
Y_ets_ts = pd.read_pickle("h1_AutoETSbase.pkl")
uidds_ts = pd.read_pickle("h1_AutoETSbase.pkl")[["unique_id",'ds']]
uidds_tr = pd.read_pickle("h1_AutoETSfitt.pkl")[["unique_id",'ds','y']]
Y_lgbm_ts = pd.concat([uidds_ts.reset_index(drop=True),
                       pd.read_pickle("h1_LGBMbase_recursive.pkl").reset_index(drop=True)],axis = 1)

Y_lgbm_tr = pd.concat([uidds_tr.reset_index(drop=True), 
                       pd.read_pickle("h1_LGBMfitt.pkl")],axis=1)

Y_lgbm_tr.columns = ["unique_id",'ds','y','LGBMRegressor']
Y_lgbm_ts.columns = ["unique_id",'ds','LGBMRegressor']

y = pd.read_pickle("etsdata.pkl")
y_hist  = y[y['ds']<=1913]
y_ts2   = y[y['ds']>1913]

S = pd.read_pickle("S_df.pkl")
tags = pd.read_pickle("tags.bin")

Wall time: 25.2 s


In [72]:
tr_ets_bu

,unique_id,ds,y,AutoETS,AutoETS/BottomUp
0,TOTAL,1,32631.0,26503.482422,28943.298660
1,TOTAL,2,31749.0,26726.798828,28959.993469
2,TOTAL,3,23783.0,26909.833984,29009.894816
3,TOTAL,4,25412.0,26795.876953,27865.213450
4,TOTAL,5,19146.0,26745.441406,27576.178188
...,...,...,...,...,...
81952915,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1909,0.0,0.079155,0.079155
81952916,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1910,0.0,0.078329,0.078329
81952917,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1911,0.0,0.077508,0.077508
81952918,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1912,0.0,0.076693,0.076693


In [6]:
%%time
# Reconcile the base predictions
bottomup = [
    BottomUp(),
    #TopDown("proportion_averages"),#'average_proportion'
    #MinTrace("mint_shrink",nonnegative = True )
]
bu = HierarchicalReconciliation(reconcilers=bottomup)
#hrec.fit(Y_df = Y_df, Y_hat_df = Y_hat_df)
h1_lgb_bu = bu.reconcile(Y_hat_df=Y_lgbm_ts, S=S, tags=tags)#,Y_df=Y_lgbm_tr)
h1_ets_bu = bu.reconcile(Y_hat_df=Y_ets_ts, S=S, tags=tags)#,Y_df=Y_ets_tr)

tr_lgb_bu = bu.reconcile(Y_hat_df=Y_lgbm_tr, S=S, tags=tags)
tr_ets_bu = bu.reconcile(Y_hat_df=Y_ets_tr, S=S, tags=tags)

tr_y = bu.reconcile(Y_hat_df=y_hist, S=S, tags=tags)
ts_y = bu.reconcile(Y_hat_df=y_ts2, S=S, tags=tags)

Wall time: 14min 19s


In [21]:
tr_lgb_bu = bu.reconcile(Y_hat_df=Y_lgbm_tr, S=S, tags=tags)
tr_ets_bu = bu.reconcile(Y_hat_df=Y_ets_tr, S=S, tags=tags)

In [22]:
tr_lbs = tr_lgb_bu["LGBMRegressor"]
tr_ebs = tr_ets_bu["AutoETS"]

In [75]:
h1lgb_bs = h1_lgb_bu['LGBMRegressor']
h1ets_bs = h1_ets_bu['AutoETS']
h1ets_bs.to_pickle("latest_etsbase.pkl")
h1lgb_bs.to_pickle("latest_lgbbase.pkl")

h1lgb_bu = h1_lgb_bu['LGBMRegressor/BottomUp']
h1ets_bu = h1_ets_bu['AutoETS/BottomUp']

ts_y.to_pickle("future_28.pkl")
tr_y.to_pickle("past_1913.pkl")
# trainning bottom
tr_lbu = tr_lgb_bu['LGBMRegressor/BottomUp']
tr_ebu = tr_ets_bu['AutoETS/BottomUp']
tr_lbs = tr_lgb_bu["LGBMRegressor"]
tr_ebs = tr_ets_bu["AutoETS"]

#del ts_y, tr_y,h1_lgb_bu, h1_ets_bu

In [15]:
%%time
# Reconcile the base predictions
mintrace = [
    #BottomUp(),
    #TopDown("proportion_averages"),#'average_proportion'
    MinTrace("mint_shrink" )
]
mint = HierarchicalReconciliation(reconcilers=mintrace)
h1_lgb_mint = mint.reconcile(Y_hat_df= Y_lgbm_ts, S=S, tags=tags, Y_df=Y_lgbm_tr)
h1_ets_mint = mint.reconcile(Y_hat_df= Y_ets_ts,  S=S, tags=tags, Y_df=Y_ets_tr)
h1lgb_mint = h1_lgb_mint.iloc[:,-1]
h1ets_mint = h1_ets_mint.iloc[:,-1]

tr_lgb_mint = mint.reconcile(Y_hat_df= Y_lgbm_tr[['unique_id','ds','LGBMRegressor']],
                             S=S, tags=tags, Y_df=Y_lgbm_tr)
tr_ets_mint = mint.reconcile(Y_hat_df= Y_ets_tr[['unique_id','ds','AutoETS']], 
                             S=S, tags=tags, Y_df=Y_ets_tr)

tr_lmint = tr_lgb_mint.iloc[:,-1]
tr_emint = tr_ets_mint.iloc[:,-1]

#del h1_lgb_mint,h1_ets_mint

Wall time: 1h 18min 13s


In [78]:
tr_lmint = tr_lgb_mint.iloc[:,-1]
tr_emint = tr_ets_mint.iloc[:,-1]

In [79]:
tr_lmint

0           32243.677858
1           32036.402240
2           23923.969539
3           25580.088159
4           19815.091269
                ...     
81952915        0.044117
81952916        0.044762
81952917        0.071843
81952918        0.058769
81952919        0.054768
Name: LGBMRegressor/MinTrace_method-mint_shrink, Length: 81952920, dtype: float64

# TD

In [15]:
path  = 'C:/PycharmProjects/pythonProject1/yjz/Extension for hts/JayCode/Data/'
h1ets_bs = pd.read_pickle("latest_etsbase.pkl")
h1lgb_bs = pd.read_pickle("latest_lgbbase.pkl")
'''
Y_lgbm_tr = pd.concat([uidds_tr.reset_index(drop=True), 
                       pd.read_pickle("h1_LGBMfitt.pkl")],axis=1)'''

y      = pd.read_pickle("thepast_1913.pkl")#['y']


In [44]:
def topdown(tr:pd.DataFrame, ts:pd.DataFrame, td_l = [], n:int = 42840, gap1:int = 1913, gap2:int = 28):
    import numpy as np
    from tqdm import tqdm
    tr = np.array(tr)
    tot = np.sum(tr[:gap1])
    fcst= ts[:gap2]
    for i in tqdm(range(n)):
        bot = sum(tr[(i)*gap1:(i+1)*gap1])
        ap  = np.mean(bot/tot) # average propotion
        td_l.append(pd.DataFrame({"0":ap*fcst}))
    return pd.concat(td_l)

In [45]:
h1ets_td = topdown(tr = y, ts = h1ets_bs, td_l = [])
h1lgb_td = topdown(tr = y, ts = h1lgb_bs, td_l = [])
tr_ltd = topdown(tr = y, ts = tr_lbs, td_l = [], gap2 = 1913)
tr_etd = topdown(tr = y, ts = tr_ebs, td_l = [], gap2 = 1913)

100%|████████████████████████████████████████████████████████████████████████████| 42840/42840 [00:15<00:00, 2821.37it/s]


# TDFP

In [2]:
import pandas as pd
import numpy as np
ts = pd.read_pickle('past_1913.pkl')
lgbbs = pd.read_pickle('fitts_base2cohe.pkl').iloc[:,0]
etsbs = pd.read_pickle('fitts_base2cohe.pkl').iloc[:,4]
from tqdm import tqdm
from functools import lru_cache
#df = pd.concat([ts,lgbs],axis=1)
lp = {"l1" :(0,1),
      "l2" :(1,(1+3)),
      "l4" :((1+3),(1+3+3)),
      "l3" :((1+3+3),(1+3+3+10)),
      "l5" :((1+3+10+3),(1+3+10+3+7)),
      "l6" :((1+3+10+3+7),(1+3+10+3+7+9)),
      "l7" :((1+3+10+3+7+9),(1+3+10+3+7+9+21)),
      "l8" :((1+3+10+3+7+9+21),(1+3+10+3+7+9+21+30)),
      "l10":((1+3+10+3+7+9+21+30),(1+3+10+3+7+9+21+30+3049)),
      "l9" :((1+3+10+3+7+9+21+30+3049),(1+3+10+3+7+9+21+30+3049+70)),
      "l11":((1+3+10+3+7+9+21+30+70+3049),(1+3+10+3+7+9+21+30+70+3049+9147)),
      "l12":((1+3+10+3+7+9+21+30+70+3049+9147),(1+3+10+3+7+9+21+30+70+3049+9147+30490))}
lvls = []
#n = 1
for i in lp.keys():
    a,b = lp[i]
    lvls.append([i[1:]]*1913*(b-a))

levels = pd.concat([pd.DataFrame({'levels':lvls[i]}) for i in range(len(lvls))]).reset_index(drop=True)
lgb = pd.concat([levels, ts,lgbbs],axis=1)
ets = pd.concat([levels, ts,etsbs],axis=1)

def tdfp(df, col2: str, col1='unique_id', col3='ds'):
    states = ['CA', 'TX', 'WI']
    categories = ['HOBBIES', 'HOUSEHOLD', 'FOODS']
    df_ = df.copy()
    splits = df_[col1].str.rsplit("/", n=1, expand=True)
    df_["parent_id"] = splits[0]
    df_["child_id"] = splits[1].fillna('')
    df_['group_sum'] = df_.groupby([col3, "parent_id", "levels"])[col2].transform("sum")
    df_["prop_to_parent"] = df_[col2] / df_['group_sum']
    for subset in [states, categories]:
        cond = (df_["parent_id"] == 'TOTAL') & (df_["child_id"].isin(subset))
        group_sum = df_.loc[cond].groupby([col3, "parent_id", "levels"])[col2].transform("sum")
        df_.loc[cond, 'group_sum'] = group_sum
        df_.loc[cond, "prop_to_parent"] = df_.loc[cond, col2] / group_sum
    condition = (df_["parent_id"] == 'TOTAL') & (df_["child_id"] == '')
    df_.loc[condition, 'group_sum'] = df_.loc[condition, col2]
    df_.loc[condition, "prop_to_parent"] = 1.0
    return df_

In [6]:
lgb.iloc[4*1913:5*1913,:]

,levels,unique_id,ds,y,lgb_base
7652,4,TOTAL/FOODS,1,23178,8607.206887
7653,4,TOTAL/FOODS,2,22758,21134.851322
7654,4,TOTAL/FOODS,3,17174,16654.912672
7655,4,TOTAL/FOODS,4,18878,18808.335524
7656,4,TOTAL/FOODS,5,14603,15549.747721
...,...,...,...,...,...
9560,4,TOTAL/FOODS,1909,23632,23478.762667
9561,4,TOTAL/FOODS,1910,23317,23210.991201
9562,4,TOTAL/FOODS,1911,26704,26817.243528
9563,4,TOTAL/FOODS,1912,31927,32030.965242


In [20]:
ts["unique_id"].unique()

array(['TOTAL', 'TOTAL/CA', 'TOTAL/TX', ...,
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_514',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_515',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_516'],
      dtype=object)

In [7]:
proportion = []
df = lgb
total = df.iloc[:1913,-1]
for ds in tqdm(range(1, 1914)):
    df2 = tdfp(df[df['ds'] == ds], col2 = df.columns[-1])
    df_ds = df2[df2['ds'] == ds].copy()
    
    parent_map = dict(zip(df_ds["unique_id"], df_ds["parent_id"]))
    prop_to_parent = dict(zip(df_ds["unique_id"], df_ds["prop_to_parent"]))
    
    @lru_cache(maxsize=None)
    def get_global_prop(node):
        if parent_map[node] == node:  
            return 1.0
        return get_global_prop(parent_map[node]) * prop_to_parent[node]
    
    df_ds["prop_global"] = df_ds["unique_id"].map(get_global_prop)
    df_ds["y_coherent"] = df_ds["prop_global"] * total[ds-1]
    proportion.append(df_ds)
    get_global_prop.cache_clear()
tdfp_lgb = pd.concat(proportion).sort_index()

proportion = []
df = ets
total = df.iloc[:1913,-1]
for ds in tqdm(range(1, 1914)):
    df2 = tdfp(df[df['ds'] == ds], col2 = df.columns[-1])
    df_ds = df2[df2['ds'] == ds].copy()
    
    parent_map = dict(zip(df_ds["unique_id"], df_ds["parent_id"]))
    prop_to_parent = dict(zip(df_ds["unique_id"], df_ds["prop_to_parent"]))
    
    @lru_cache(maxsize=None)
    def get_global_prop(node):
        if parent_map[node] == node:  
            return 1.0
        return get_global_prop(parent_map[node]) * prop_to_parent[node]
    
    df_ds["prop_global"] = df_ds["unique_id"].map(get_global_prop)
    df_ds["y_coherent"] = df_ds["prop_global"] * total[ds-1]
    proportion.append(df_ds)
    get_global_prop.cache_clear()
tdfp_ets = pd.concat(proportion).sort_index()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1913/1913 [16:53<00:00,  1.89it/s]


In [11]:
lgb

,levels,unique_id,ds,y,lgb_base
0,1,TOTAL,1,32631,20891.244171
1,1,TOTAL,2,31749,31177.044432
2,1,TOTAL,3,23783,24506.852209
3,1,TOTAL,4,25412,25270.598233
4,1,TOTAL,5,19146,20683.996429
...,...,...,...,...,...
81952915,12,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1909,0,0.125571
81952916,12,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1910,0,0.134732
81952917,12,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1911,0,0.139261
81952918,12,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1912,0,0.150921


In [14]:
fitt = pd.read_pickle('fitts_base2cohe.pkl')
fitt.loc[:,'lgb_td'] = tdfp_lgb.iloc[:,-1]
fitt.loc[:,'ets_td'] = tdfp_ets.iloc[:,-1]
fitt.to_pickle("fitts_base2cohe_tdfp.pkl")

# Saver

In [63]:
tr_lbu

,0
0,42166.539062
1,42043.816406
2,41803.699219
3,41535.496094
4,41288.902344
...,...
23,0.162141
24,0.161249
25,0.160373
26,0.160318


In [82]:
#h1_ets_td = topdown(y, h1ets_ts, td_l = [])
#h1_lgb_td = topdown(y, h1lgb_ts, td_l = [])

allfcst = pd.DataFrame({"lgb_base": h1lgb_bs,
                        "lgb_bu":   h1lgb_bu,
                        "lgb_td":   h1lgb_td['0'].reset_index(drop=True),
                        "lgb_mint": h1lgb_mint,
                        "ets_base": h1ets_bs,
                        "ets_bu":   h1ets_bu,
                        "ets_td":   h1ets_td['0'].reset_index(drop=True),
                        "ets_mint": h1ets_mint})
#allfcst.to_pickle("fcsts_base2cohe.pkl")

allfitt = pd.DataFrame({"lgb_base": tr_lbs,
                        "lgb_bu":   tr_lbu,
                        "lgb_td":   tr_ltd['0'].reset_index(drop=True),
                        "lgb_mint": tr_lmint,
                        "ets_base": tr_ebs,
                        "ets_bu":   tr_ebu,
                        "ets_td":   tr_etd['0'].reset_index(drop=True),
                        "ets_mint": tr_emint})
allfcst.to_pickle("fcsts_base2cohe.pkl")
allfitt.to_pickle("fitts_base2cohe.pkl")

In [83]:
allfitt

,lgb_base,lgb_bu,lgb_td,lgb_mint,ets_base,ets_bu,ets_td,ets_mint
0,20891.244171,62305.720660,20891.244171,32243.677858,26503.482422,28943.298660,26503.482422,30779.641970
1,31177.044432,50048.874233,31177.044432,32036.402240,26726.798828,28959.993469,26726.798828,30097.092005
2,24506.852209,27370.797167,24506.852209,23923.969539,26909.833984,29009.894816,26909.833984,27827.112893
3,25270.598233,24096.944690,25270.598233,25580.088159,26795.876953,27865.213450,26795.876953,24061.809936
4,20683.996429,18681.959903,20683.996429,19815.091269,26745.441406,27576.178188,26745.441406,25925.486709
...,...,...,...,...,...,...,...,...
81952915,0.125571,0.125571,0.131582,0.044117,0.079155,0.079155,0.157191,0.015872
81952916,0.134732,0.134732,0.131563,0.044762,0.078329,0.078329,0.156265,0.045219
81952917,0.139261,0.139261,0.151282,0.071843,0.077508,0.077508,0.155332,0.074854
81952918,0.150921,0.150921,0.183255,0.058769,0.076693,0.076693,0.155178,0.070326


In [69]:
tr_lbu

0          37929.870202
1          36220.831410
2          34798.572205
3          34825.058046
4          39600.392103
               ...     
1199515        0.177103
1199516        0.186536
1199517        0.174125
1199518        0.242970
1199519        0.142334
Name: LGBMRegressor/BottomUp, Length: 1199520, dtype: float64